In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [3]:
# write your dataset directory path
data_dir = '/kaggle/input/real-life-violence/Real life violence dataset'
batch_size = 32
image_width = 224
image_height = 224


In [4]:
# Create ImageDataGenerator for training data with validation split
train_data_augmentor = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.05,
    horizontal_flip=True,
    validation_split=0.2
)

In [5]:
# Load training data from directory
train_data = train_data_augmentor.flow_from_directory(
    data_dir,
    target_size=(image_width, image_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

Found 11245 images belonging to 2 classes.


In [6]:
# Load validation data from directory
validation_data = train_data_augmentor.flow_from_directory(
    data_dir,
    target_size=(image_width, image_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

Found 2811 images belonging to 2 classes.


In [7]:
# Check class indices
print(train_data.class_indices)

{'nonviolence': 0, 'violence': 1}


In [9]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2

# Path to the MobileNetV2 weights file
weights_path = "/kaggle/input/mobilenetv2-h5/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5"

# Load MobileNetV2 pre-trained on ImageNet without including the top classification layer
base_model = MobileNetV2(weights=weights_path, include_top=False, input_shape=(image_width, image_height, 3))

# Freeze the base model's layers so they are not trainable
for layer in base_model.layers:
    layer.trainable = False

# Add additional layers on top of the base model
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=output)


In [10]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
# Define number of epochs
epochs = 8

# Train the model with augmented data
history = model.fit(
    train_data,
    verbose=1,
    epochs=epochs,
    validation_data=validation_data,
)

Epoch 1/8


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


352/352 ━━━━━━━━━━━━━━━━━━━━ 400s 1s/step - accuracy: 0.8315 - loss: 0.4104 - val_accuracy: 0.8844 - val_loss: 0.3275
Epoch 2/8
352/352 ━━━━━━━━━━━━━━━━━━━━ 416s 1s/step - accuracy: 0.9344 - loss: 0.1679 - val_accuracy: 0.8300 - val_loss: 0.4344
Epoch 3/8
352/352 ━━━━━━━━━━━━━━━━━━━━ 383s 1s/step - accuracy: 0.9507 - loss: 0.1286 - val_accuracy: 0.8634 - val_loss: 0.3772
Epoch 4/8
352/352 ━━━━━━━━━━━━━━━━━━━━ 377s 1s/step - accuracy: 0.9548 - loss: 0.1137 - val_accuracy: 0.8723 - val_loss: 0.3912
Epoch 5/8
352/352 ━━━━━━━━━━━━━━━━━━━━ 384s 1s/step - accuracy: 0.9631 - loss: 0.1006 - val_accuracy: 0.8648 - val_loss: 0.3599
Epoch 6/8
352/352 ━━━━━━━━━━━━━━━━━━━━ 365s 1s/step - accuracy: 0.9639 - loss: 0.0906 - val_accuracy: 0.8613 - val_loss: 0.4068
Epoch 7/8
352/352 ━━━━━━━━━━━━━━━━━━━━ 382s 1s/step - accuracy: 0.9697 - loss: 0.0714 - val_accuracy: 0.8321 - val_loss: 0.5131
Epoch 8/8
352/352 ━━━━━━━━━━━━━━━━━━━━ 382s 1s/step - accuracy: 0.9740 - loss: 0.0630 - val_accuracy: 0.8801 - val

In [14]:
from tensorflow.keras.models import save_model
from IPython.display import FileLink

# Save the model with .h5 extension
model_path = "./bablu_model.h5"
model.save(model_path)

# Create a download link for the model
FileLink(model_path)



/kaggle/working/bablu_model.h5